# Geometry calibration notebook.
## Use this after ADU_Thresh notebook and setting proper ADU threshes for the azav and the the Jungfrau Sum. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from xrayscatteringtools.io import combineRuns, get_data_paths
from xrayscatteringtools.utils import enable_underscore_cleanup, azimuthalBinning
from xrayscatteringtools.plotting import plot_jungfrau
from xrayscatteringtools.calib.geometry_calibration import run_geometry_calibration
enable_underscore_cleanup()

### Importing data. Edit the run number, you may need to edit some of the keys. Usually is should work find so long as there is nothing special going on.
The main key that we work with is the `'Sums/jungfrau4M_calib_threshADU'` key. You can define your own in the producer with the `sumAlgo` kwarg of `det.storeSum()` and appropriate kwargs to `det.processSums()`. The key takeway is that any number after `'threshADU'` will be a lower bound in ADU for thresholding.

In [ ]:
###############################################
runNumbers = []
folders = get_data_paths(runNumbers) # Defaults to the info in config.yaml. You can overwrite this with strings, character arrays, or lists of either.
###############################################
# (1) keys_to_combine: some keys loaded for each shot & stored per shot 
# (2) keys_to_sum: some keys loaded per each run and added 
# (3) keys_to_check : check if some keys exits and have same values in all runs and load these keys 
_keys_to_combine = [
    'ebeam/photon_energy',
]
_keys_to_sum = [
    'Sums/jungfrau4M_calib_thresADU5', # <- This key may need to be changed to reflect the ADU threshold set in the producer
    # 'Sums/jungfrau4M_calib_dg2low_threshADU5' # <- Any user-defined sums can go here too.
]
_keys_to_check = [
    # 'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask', # <- Any masks for different azav regions can go here. We will combine them in the notebook
    # 'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask',
    'UserDataCfg/jungfrau4M/cmask', # <- Combined mask for this run stored in the datastream. Ususally a combination of a linemask and any pixel masks.
    'UserDataCfg/jungfrau4M/x',
    'UserDataCfg/jungfrau4M/y',
]
##### Load the data in #####
_data = combineRuns(runNumbers, folders, _keys_to_combine, _keys_to_sum, _keys_to_check, verbose=False)  # this is the function to load the data with defined keys
############################

# userMaskFiltered = _data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask'].astype(bool) # User mask for this region
# userMask = _data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask'].astype(bool) # User mask for this region
jungfrau_sum = _data['Sums/jungfrau4M_calib_thresADU5']  # Total Jungfrau detector counts summed in a run
x, y = _data['UserDataCfg/jungfrau4M/x'], _data['UserDataCfg/jungfrau4M/y'] # coordinates of Jungfrau detector pixels x,y in micron
cmask = _data['UserDataCfg/jungfrau4M/cmask'].astype(bool) # Standard Calibration mask: Dark + Ped + Line
xray_keV = _data['ebeam/photon_energy'] # x-ray energy energy in keV

## Now that the data is loaded, let's prep the mask.


In [ ]:
############################
mask = cmask # & usermask 
_binary_dilation_amount = 3
############################
# Preforming a binary opening to mask out groups of bad pixels together.
if _binary_dilation_amount:
    mask = ndi.binary_opening(mask, structure=np.ones((1, _binary_dilation_amount, _binary_dilation_amount), dtype=bool)) & mask
plt.figure(figsize=[14,12])
plt.subplot(2,2,1)
plot_jungfrau(y,-x,cmask)
plt.title('cmask')
plt.subplot(2,2,2)
plot_jungfrau(y,-x,mask)
plt.title('Final Mask')
plt.subplot(2,2,3)
pcm = plot_jungfrau(y,-x,jungfrau_sum)
plt.colorbar(pcm)
plt.title('J4M Sum')
plt.subplot(2,2,4)
pcm = plot_jungfrau(y,-x,jungfrau_sum*mask)
plt.colorbar(pcm)
plt.title('Masked J4M Sum')
plt.show()
print(f'Percentage Masked (cmask): {np.sum(~cmask)/np.size(cmask):.2f}%')
print(f'Percentage Masked (final ask): {np.sum(~mask)/np.size(mask):.2f}%')

### One nice big plot to show the final image (masked) before geometry optimization

In [ ]:
plt.figure(figsize=[18,14])
_jungfrau_sum = np.copy(jungfrau_sum)
_jungfrau_sum[~mask] = np.nan
pcm = plot_jungfrau(y,-x,_jungfrau_sum)
plt.colorbar(pcm)
plt.title('Masked Jungfrau Sum (big)')
plt.show()

### Importing or loading the theory pattern to fit this data to.
For data inside the xrayscatteringtools module, the docstring will contain more information about that pattern. `sf6?`

In [ ]:
###################################################
from xrayscatteringtools.theory.patterns import sf6
theory_q, theory_I_q = sf6.q, sf6.I_q
###################################################
plt.plot(theory_q,theory_I_q)
plt.title('Theory Pattern')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.ylabel(r'I(q) ($e^2$)')
plt.show()

### Now to run the geometry calibration.
`run_geometry_calibration()` Has many parameters which are not used here. Type `run_geometry_calibration?` for more information.

In [ ]:
##################################################
keV = xray_keV
# keV = 9.666 # Override for photon energy if needed
##################################################
fit, popt, pcov = run_geometry_calibration(
    jungfrau_sum,
    x,
    y,
    mask,
    theory_q,
    theory_I_q,
    9.666)
print(f'Converged Parameters: \n amplitude: {popt[0]:.2f}\n x0: {popt[1]:.2f}\n y0: {popt[2]:.2f}\n z0: {popt[3]:.2f}')

## Taking a look at the resultant fit, comparing to theory.

In [ ]:
# Setting up a dict for cleaner usage.
_kwargs = {
    'x': x,
    'y': y,
    'x0': popt[1],
    'y0': popt[2],
    'z0': popt[3],
    'mask': ~mask,
    'keV': keV,
}
q, azav_exp = azimuthalBinning(jungfrau_sum,**_kwargs)
q, azav_fit = azimuthalBinning(fit,**_kwargs)
with np.errstate(invalid='ignore'):
    pdiff = 100*(azav_exp-azav_fit)/azav_fit
plt.figure(figsize=[18,5])
plt.subplot(1,3,1)
plt.plot(q,azav_exp,label='Experiment')
plt.plot(q,azav_fit,label='Fit')
plt.legend()
plt.title('Fitted Scattering Pattern')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.ylabel(r'I(q) ($e^2$)')
plt.subplot(1,3,2)
plt.plot(q,azav_exp*q,label='Experiment')
plt.plot(q,azav_fit*q,label='Fit')
plt.legend()
plt.title('Fitted Scattering Pattern')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.ylabel(r'I(q)*q ($e^2 \AA^{-1}$)')
plt.subplot(1,3,3)
plt.plot(q,pdiff)
plt.title('Percent Difference')
plt.ylabel('%')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.show()

### Looking at the difference between the full images:

In [ ]:
#############
_vlimsPdiff = 20
#############
pdiff_img = 100*(jungfrau_sum-fit)/fit
pdiff_img[~mask] = np.nan
plt.figure(figsize=[18,5])
plt.subplot(1,3,1)
_jungfrau_sum = np.copy(jungfrau_sum)
_jungfrau_sum[~mask] = np.nan
pcm = plot_jungfrau(y,-x,_jungfrau_sum)
plt.colorbar(pcm)
plt.title('Experiment')
plt.subplot(1,3,2)
_fit = np.copy(fit)
_fit[~mask] = np.nan
pcm = plot_jungfrau(y,-x,_fit)
plt.colorbar(pcm)
plt.title('Fit')
plt.subplot(1,3,3)
pcm=plot_jungfrau(y,-x,pdiff_img,vmin=-_vlimsPdiff,vmax=_vlimsPdiff,cmap='RdBu')
plt.colorbar(pcm)
plt.title('Percent Difference Image')
plt.show()

### Remember, when pasting the x0, y0, and z0 parameters into the producer, the z0 parameters should be in mm, no micron. Below are the pasteable correct parameters:

In [ ]:
print(f"func_dict['center'] = [{popt[1]}, {popt[2]}]")
print(f"func_dict['dis_to_sam'] = {popt[3]/1000}")